In [1]:
import jsonlines
from transformers import *
import torch
import numpy as np

In [2]:
FILE_TEST_TRUE = "scicite/data/acl-arc/test.jsonl"
FILE_DEV_TRUE = "scicite/data/acl-arc/dev.jsonl"
FILE_TRAIN_TRUE = "scicite/data/acl-arc/train.jsonl"

FILE_TEST_NEW = "scicite/data/acl-arc-emb/test.jsonl"
FILE_DEV_NEW = "scicite/data/acl-arc-emb/dev.jsonl"
FILE_TRAIN_NEW = "scicite/data/acl-arc-emb/train.jsonl"

FILE_TRUE_EMBS = "scicite/data/acl-arc-emb/embeddings.npy"



FILE_TRAIN_TRUE_SCAF_1 = "scicite/data/acl-arc/scaffolds/cite-worthiness-scaffold-train.jsonl"
FILE_TRAIN_TRUE_SCAF_1_NEW = "scicite/data/acl-arc-emb/scaffolds/cite-worthiness-scaffold-embs/"

FILE_TRAIN_TRUE_SCAF_2 = "scicite/data/acl-arc/scaffolds/sections-scaffold-train.jsonl"
FILE_TRAIN_TRUE_SCAF_2_NEW = "scicite/data/acl-arc-emb/scaffolds/sections-scaffold-embs/"

Bert

In [3]:
tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased",do_lower_case=True)

In [4]:
model = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
def check_item(true_file):
    with jsonlines.open(true_file, 'r') as reader:
        for item in enumerate(reader):
            print(item)
            break

In [6]:
check_item(FILE_TRAIN_TRUE_SCAF_1)

(0, {'text': 'Texts are often structured into segments to ease understanding and readability of texts .', 'citing_paper_id': 'N12-1064', 'cited_paper_id': None, 'citing_paper_year': 2012, 'cited_paper_year': -1, 'citing_paper_title': 'a holistic lexiconbased approach to opinion mining', 'cited_paper_title': None, 'cited_author_ids': None, 'citing_author_ids': None, 'extended_context': None, 'section_number': 0, 'section_title': None, 'intent': None, 'cite_marker_offset': None, 'sents_before': None, 'sents_after': None, 'cleaned_cite_text': 'Texts are often structured into segments to ease understanding and readability of texts .', 'citation_id': None, 'citation_excerpt_index': 0, 'section_name': '1 Introduction', 'is_citation': False})


In [7]:
check_item(FILE_TRAIN_TRUE_SCAF_2)

(0, {'text': 'For topic modeling , we use the widely applied LDA ( Blei et al. , 2003 ) .', 'section_name': 'related work', 'citing_paper_id': 'N12-1064', 'cited_paper_id': 'External_351'})


In [10]:
def save_embs():
    emb_cnt = 0
    embeddings = []
    
    with jsonlines.open(FILE_TEST_TRUE, 'r') as reader:
        with jsonlines.open(FILE_TEST_NEW, 'w') as writer:
            for item in enumerate(reader):
                tok_out = tokenizer.encode(item[1]['text'], padding='max_length', max_length=400)
                tok_out = torch.tensor(tok_out).unsqueeze(0)
                #print(tok_out.size())
                emb_out = model(tok_out)[0]
                
                item[1]['emb_id'] = emb_cnt
                embeddings.append(emb_out.detach().numpy())
                writer.write(item[1])
                emb_cnt += 1
    print("done_test")
    
    with jsonlines.open(FILE_DEV_TRUE, 'r') as reader:
        with jsonlines.open(FILE_DEV_NEW, 'w') as writer:
            for item in enumerate(reader):
                tok_out = tokenizer.encode(item[1]['text'], padding='max_length', max_length=400)
                emb_out = model(torch.tensor(tok_out).unsqueeze(0))[0]
                
                item[1]['emb_id'] = emb_cnt
                embeddings.append(emb_out.detach().numpy())
                writer.write(item[1])
                emb_cnt += 1
    print("done_dev")
                
    with jsonlines.open(FILE_TRAIN_TRUE, 'r') as reader:
        with jsonlines.open(FILE_TRAIN_NEW, 'w') as writer:
            for item in enumerate(reader):
                tok_out = tokenizer.encode(item[1]['text'], padding='max_length', max_length=400)
                emb_out = model(torch.tensor(tok_out).unsqueeze(0))[0]
                
                item[1]['emb_id'] = emb_cnt
                embeddings.append(emb_out.detach().numpy())
                writer.write(item[1])
                emb_cnt += 1
    print("done_train")
                
    np.save(FILE_TRUE_EMBS, np.array(embeddings))

In [ ]:
save_embs()

done_test
done_dev


In [12]:
with jsonlines.open(FILE_TEST_TRUE, 'r') as reader:
    for item in enumerate(reader):
        tok_out_1 = tokenizer.encode(item[1]['text'], padding='max_length', max_length=400)
        break

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
tok_out = np.expand_dims(np.array(tok_out), axis=0)
            emb_out = model(tok_out)[0]

In [4]:
rewrite(FILE_TEST_TRUE, FILE_TEST_TRUE_NEW, 10)

In [5]:
rewrite(FILE_DEV_TRUE, FILE_DEV_TRUE_NEW, 10)

In [6]:
rewrite(FILE_TRAIN_TRUE, FILE_TRAIN_TRUE_NEW, 100)

In [9]:
rewrite(FILE_TRAIN_TRUE_SCAF_1, FILE_TRAIN_TRUE_SCAF_1_NEW, 200)

In [10]:
rewrite(FILE_TRAIN_TRUE_SCAF_2, FILE_TRAIN_TRUE_SCAF_2_NEW, 200)

In [8]:
with jsonlines.open(FILE_DEV_TRUE, 'r') as reader:
    for i, entry in enumerate(reader):
        buf = entry

In [12]:
list(buf.keys())

['text',
 'citing_paper_id',
 'cited_paper_id',
 'citing_paper_year',
 'cited_paper_year',
 'citing_paper_title',
 'cited_paper_title',
 'cited_author_ids',
 'citing_author_ids',
 'extended_context',
 'section_number',
 'section_title',
 'intent',
 'cite_marker_offset',
 'sents_before',
 'sents_after',
 'cleaned_cite_text',
 'citation_id',
 'citation_excerpt_index',
 'section_name']

In [13]:
buf['text']

'This result is consistent with other works using this model with these features ( Andrews et al. , 2009 ; Silberer and Lapata , 2012 ) .'

In [19]:
buf['extended_context']

'The 2D models employing feature norms and association norms do significantly better than the text-only model ( two-tailed t-test ) . This result is consistent with other works using this model with these features ( Andrews et al. , 2009 ; Silberer and Lapata , 2012 ) . We also see that the SURF visual words are able to provide notable , albeit not significant , improvements over the text-only modality .'

In [20]:
buf['citation_excerpt_index']

0